In [175]:
import numpy as np
import scipy.stats 
import gym
import random
# create a class for our policy gradient
class PolicyG(object):
    # initalize our variables with constructor
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.theta=np.random.random(4)
        self.alpha=0.01
        self.gamma=0.99
        self.actions=[]
        self.probs=[]
        self.states=[]
        self.rewards=[]
        
    # inconjunction with the variabler names outputed by Gym env
    ## append our data each loop
    def memory(self,state,action,prob,reward):
        
        self.states.append(state)
        self.actions.append(action)
        self.probs.append(prob)
        self.rewards.append(reward)
        
    def sigmoid(self,x):
        return (1)/(1+np.exp(-x))
    
    def prob(self,x):
        # prob_0 is the model pi_0 like a logisitc regression model where theta is the parameter
        # and x is the data input in this action it is action array from the gym virtual env
        #trivially it is p, and 1-p by laws of prob
        y=x@self.theta
        prob0=self.sigmoid(y)
        return np.array([prob0,1-prob0])
    # sample action from our dist
    
    def act(self,x):
        # we sample from our logisitc regression function
        prob1=self.prob(x)
        action=np.random.choice([0,1],p=prob1)
        return action, prob1[action]
    
    def grad_log(self,x):
        #print(self.actions)
        print(self.theta)
        #print(x[0])
        y=x@self.theta 
        grad_p0=np.array(x)-np.array(x)*self.sigmoid(y)
        grad_p1=-np.array(x)*self.sigmoid(y)
        
        return grad_p0,grad_p1
    
    
    # gradient dotted with reward of action in a single episode 
    #def grad_log_dot_reward(self,grad_log,discounted_rewards):
      #  return grad_log.T@discounted_rewards
    
    def discount_rewards(self):
        
        discounted_rewards=np.zeros(len(self.rewards))
        
        cum_rewards=0
        for i in reversed(range(0,len(self.rewards))):
            ## past events matter less by gamma param
            cum_rewards= cum_rewards +self.rewards[i]+self.gamma**(i)
            discounted_rewards[i]=cum_rewards
        return cum_rewards
    ## now  we optimize policy over trajectories and maintain the dimension of the gradient
    ## via the asset command
    
    
    def optimize(self,s,ep):
        self.rewards -= np.mean(self.rewards)
        self.rewards /= np.std(self.rewards)
        for t in range(len(self.states)):
            grad_log=self.grad_log(self.states[t])
            grad_log=np.array(grad_log)
            discounted_rewards=self.discount_rewards()
            #if len(self.theta) != len(self.actions):
            #self.theta=np.reshape(self.theta,len(self.actions))
            #print(self.theta)
            self.theta = (self.theta)+ self.alpha *grad_log*discounted_rewards
                
        # print(self.theta)
        self.states = []
        self.actions = []
        self.probs = []
        self.rewards = [] 
    def save(self):
        pass

In [176]:
import agent
env=gym.make('CartPole-v1')

state=env.reset()

score=0
epi=0
state_size=4
action_size=env.action_space.n
pol=PolicyG(state_size, action_size)

MAX_EP=1000

while epi<MAX_EP:
    actione, prob= pol.act(np.array(state))
    state,reward,done, info=env.step(actione)
    
    if done:
        reward= -10
    score+=reward
    pol.memory(state,actione,prob,reward)
    
    if done:
        epi+=1
    
        pol.optimize(state,epi)
    
        print('Episode:{} with score {}'.format(epi,score))
        score=0
        state=env.reset()
        if epi% 10 == 0:
            pol.save()

[0.986151   0.32458668 0.54916104 0.50562218]
[[0.98453059 0.34134064 0.55064097 0.48574232]
 [0.98766153 0.30896892 0.54778147 0.52415385]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 4)